In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import copy
import os
import itertools

In [3]:
import pandas as pd
import numpy as np
import networkx as nx
from scipy.stats import pearsonr
from scipy.stats import skew
from tqdm import tqdm
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
from IPython.display import display, Markdown, Latex
import joblib

In [5]:
from pathlib import Path
home = str(Path.home())

In [6]:
import sys
sys.path.insert(0,"%s/rankability_toolbox_dev"%home)
import pyrankability

In [8]:
sys.path.insert(0,"%s/marchmadness_study"%home)
import base

In [10]:
games={}
remaining_games={}
madness_teams={}
all_teams={}
years = ["2002","2003","2004","2005","2006","2007","2008","2009","2010","2011","2012","2013","2014","2015","2016","2017","2018"]
for year in years:
    games[year],remaining_games[year] = base.read_data('../data/%steams.txt'%year,'../data/%sgames.txt'%year,'../data/%sMadnessTeams.txt'%year)
    madness_teams[year] = list(np.unique(list(games[year].team1_name.loc[games[year].team1_madness == 1]) + list(games[year].team2_name.loc[games[year].team2_madness == 1])))
    all_teams[year] = list(np.unique(list(games[year].team1_name) + list(games[year].team2_name)))
print(year)
games[year]

2018


,team2,team1,notsure1,date,H_A_N1,points1,H_A_N2,points2,team1_name,team2_name,team1_madness,team2_madness
0,1,16,737011,2017-11-13,1,83,-1,69,Arkansas_St,Abilene_Chr,0,0
1,1,41,737114,2018-02-24,-1,74,1,72,Cent_Arkansas,Abilene_Chr,0,0
3,1,143,737018,2017-11-20,-1,75,1,67,Lipscomb,Abilene_Chr,1,0
4,1,143,737045,2017-12-17,1,67,-1,65,Lipscomb,Abilene_Chr,1,0
5,1,199,737056,2017-12-28,1,77,-1,74,New_Orleans,Abilene_Chr,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
5535,351,318,737048,2017-12-20,1,91,-1,74,Utah_St,Youngstown_St,0,0
5536,351,339,737086,2018-01-27,1,85,-1,67,WI_Green_Bay,Youngstown_St,0,0
5537,351,340,737084,2018-01-25,1,66,-1,55,WI_Milwaukee,Youngstown_St,0,0
5538,351,347,737074,2018-01-15,1,77,-1,67,Wright_St,Youngstown_St,1,0


In [11]:
year = '2002'

## Run Colley without weighting or ties or subsets

In [62]:
game_df = pd.DataFrame({"team1_name":games[year]['team1_name'],
                        "team1_score":games[year]['points1'],
                        "team1_H_A_N": games[year]['H_A_N1'],
                        "team2_name":games[year]['team2_name'],
                        "team2_score":games[year]['points2'],
                        "team2_H_A_N": games[year]['H_A_N1'],
                        "date": games[year]['date']
                       }).sort_values(by='date').drop('date',axis=1)

map_func = lambda linked: pyrankability.construct.colley_matrices(linked,direct_thres=0,weight_indirect=0)
colley_matrix,colley_b = pyrankability.construct.map_vectorized(game_df,map_func)
colley_ranking,colley_r,colley_perm = pyrankability.rank.ranking_from_matrices(colley_matrix.fillna(0),colley_b.fillna(0))

In [63]:
colley_ranking.sort_values()

team2
Kansas              1
Maryland            2
Duke                3
Cincinnati          4
Oklahoma            5
                 ... 
Chicago_St        323
Ark_Pine_Bluff    324
Liberty           325
Mt_St_Mary's      326
Morgan_St         327
Length: 327, dtype: int64

In [65]:
colley_r.sort_values(ascending=False)

team2
Kansas            0.548560
Maryland          0.528899
Duke              0.527267
Cincinnati        0.498952
Oklahoma          0.471128
                    ...   
Chicago_St       -0.471356
Ark_Pine_Bluff   -0.478928
Liberty          -0.479375
Mt_St_Mary's     -0.484756
Morgan_St        -0.492434
Length: 327, dtype: float64

In [66]:
colley_matrix.columns[colley_perm]

Index(['Kansas', 'Maryland', 'Duke', 'Cincinnati', 'Oklahoma', 'Alabama',
       'Illinois', 'Arizona', 'Kentucky', 'Florida',
       ...
       'Loyola_MD', 'Coppin_St', 'Nicholls_St', 'F_Dickinson', 'Savannah_St',
       'Chicago_St', 'Ark_Pine_Bluff', 'Liberty', 'Mt_St_Mary's', 'Morgan_St'],
      dtype='object', name='team2', length=327)

## Run Massey without weighting or ties or subsets

In [70]:
game_df = pd.DataFrame({"team1_name":games[year]['team1_name'],
                        "team1_score":games[year]['points1'],
                        "team1_H_A_N": games[year]['H_A_N1'],
                        "team2_name":games[year]['team2_name'],
                        "team2_score":games[year]['points2'],
                        "team2_H_A_N": games[year]['H_A_N1'],
                        "date": games[year]['date']
                       }).sort_values(by='date').drop('date',axis=1)

map_func = lambda linked: pyrankability.construct.massey_matrices(linked,direct_thres=0,weight_indirect=0)
massey_matrix,massey_b = pyrankability.construct.map_vectorized(game_df,map_func)
massey_ranking,massey_r,massey_perm = pyrankability.rank.ranking_from_matrices(massey_matrix.fillna(0),massey_b.fillna(0))

In [71]:
massey_ranking.sort_values()

team2
Duke                1
Cincinnati          2
Florida             3
Kansas              4
Maryland            5
                 ... 
Mt_St_Mary's      323
Nicholls_St       324
Morris_Brown      325
Ark_Pine_Bluff    326
Savannah_St       327
Length: 327, dtype: int64

In [74]:
massey_r.sort_values(ascending=False)

team2
Duke              29.711781
Cincinnati        24.998968
Florida           24.484710
Kansas            23.884695
Maryland          23.487082
                    ...    
Mt_St_Mary's     -23.105544
Nicholls_St      -23.345540
Morris_Brown     -25.058776
Ark_Pine_Bluff   -26.374816
Savannah_St      -26.851603
Length: 327, dtype: float64

In [76]:
massey_matrix.columns[massey_perm]

Index(['Duke', 'Cincinnati', 'Florida', 'Kansas', 'Maryland', 'Oregon',
       'Oklahoma', 'Kentucky', 'Stanford', 'Indiana',
       ...
       'Chicago_St', 'Liberty', 'Southern_Univ', 'Prairie_View', 'Long_Island',
       'Mt_St_Mary's', 'Nicholls_St', 'Morris_Brown', 'Ark_Pine_Bluff',
       'Savannah_St'],
      dtype='object', name='team2', length=327)